In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.dataset.dataset import TextDataset
import torch 
from torch import nn

/home/dzhunk/University/lillama/src/dataset/dataset.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
data = TextDataset(
    corpus_path = 'data/corpus.txt',
    json_path='data/tiny_stories',
    save_tokenizer_to = 'bpe',
    max_len = 300,
    vocab_size = 15
)
print('created train data with size:', len(data))

0it [00:00, ?it/s]

created train data with size: 20869767


In [4]:
generator = torch.Generator().manual_seed(42)
datasets = torch.utils.data.random_split(data, [0.95, 0.05], generator=generator)
train_data = datasets[0]
test_data = datasets[1]
print(len(train_data))
print(len(test_data))

19826279
1043488


In [5]:
from torch.utils.data import DataLoader
from src.model.language_model import LanguageModel
from src.dataset.dataset import Collator
from src.utils.trainer import CosineAnnealingWithWarmupLR

collate_fn = Collator(pad_value=data.pad_id)
train_loader = DataLoader(train_data, batch_size=768, shuffle=True, num_workers=4, collate_fn=collate_fn)
test_loader = DataLoader(train_data, batch_size=768, shuffle=False, num_workers=4, collate_fn=collate_fn)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LanguageModel(
    embed_dim=32,
    vocab_size=data.vocab_size,
    max_len=data.max_len,
    pad_idx=data.pad_id,
    num_layers=2,
    num_heads=2,
    dropout=0,
    feedforward_dim=32
)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
# lr_scheduler = CosineAnnealingWithWarmupLR(optimizer=optimizer, warmup_steps=50, max_steps=200)

In [6]:
print(model)

LanguageModel(
  (transformer): TransformerDecoder(
    (embedding): Embedding(15, 32, padding_idx=0)
    (pos_encoder): PositionalEncoding()
    (decoder): ModuleList(
      (0-1): 2 x DecoderBlock(
        (Q_proj): Linear(in_features=32, out_features=32, bias=True)
        (K_proj): Linear(in_features=32, out_features=32, bias=True)
        (V_proj): Linear(in_features=32, out_features=32, bias=True)
        (masked_multihead_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (lay_norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (feedforward): Sequential(
          (0): Linear(in_features=32, out_features=32, bias=True)
          (1): ReLU()
          (2): Linear(in_features=32, out_features=32, bias=True)
        )
        (dropout): Dropout(p=0, inplace=False)
        (lay_norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (linear): Lin

In [7]:
from src.utils.wandb_logger import WandbLogger
from src.utils.trainer import train

wdb = WandbLogger(
    project_name="little-lama",
    config={"loh": "loh"}
)
train(
    model, optimizer, criterion, train_loader, 
    train_loader, 100, DEVICE, wdb, 
    log_output = False, grad_clipping=10
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: dzhunkoffski. Use `wandb login --relogin` to force relogin


=== Epoch 1 ===


: 

итоговый ран из wandb